In [1]:
from __future__ import unicode_literals, print_function

import random
from pathlib import Path
import spacy
from tqdm import tqdm
from spacy.training import Example
import pickle

In [3]:
TRAIN_DATA = [
    ("Check the south facade of the building", {"entities": [(10, 15, "DIRECTION"), (30, 38, "BUILDING")]}),
    ("I wanna see how is the north side of the stadium look like", {"entities": [(23, 28, "DIRECTION"), (41, 48, "BUILDING")]}),
    ("Boss wanna see how is the west side of the hospital look like", {"entities": [(26, 30, "DIRECTION"), (43, 51, "BUILDING")]}),
    ("Go east side of that tower", {"entities": [(3, 7, "DIRECTION"), (21, 26, "BUILDING")]}),
    ("Fly above the playground", {"entities": [(4, 9, "DIRECTION"), (14, 24, "BUILDING")]}),
    ("Fly above the soccer field", {"entities": [(4, 9, "DIRECTION"), (14, 26, "BUILDING")]}),
    ("Fly to the west of the basketball court", {"entities": [(11, 15, "DIRECTION"), (23, 39, "BUILDING")]}),
    ("Go to the southwest side of the bridge", {"entities": [(10, 19, "DIRECTION"), (32, 38, "BUILDING")]}),
    ("Go to the southwest side of area B", {"entities": [(10, 19, "DIRECTION"), (28, 34, "BUILDING")]}),
    ("Check out the east of D", {"entities": [(14, 18, "DIRECTION"), (22, 23, "BUILDING")]}),
    ("Fly to the north of E", {"entities": [(11, 16, "DIRECTION"), (20, 21, "BUILDING")]}),
    ("Check the northwest of Area A", {"entities": [(10, 19, "DIRECTION"), (23, 29, "BUILDING")]}),
    ("Take a screenshot on the northwest of Area A", {"entities": [(7, 17, "FUNCTION"), (25, 34, "DIRECTION"), (43, 44, "BUILDING")]}),
    ("Take a screenshot on the west of C", {"entities": [(7, 17, "FUNCTION"), (25, 29, "DIRECTION"), (33, 34, "BUILDING")]}),
    ("Go to the north of C and take a screenshot", {"entities": [(10, 15, "DIRECTION"), (19, 20, "BUILDING"), (32, 42, "FUNCTION")]}),
]

In [4]:
model = None
output_dir=Path("ner/")
n_iter=100

   #load the model

if model is not None:
    nlp = spacy.load(model)  
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')  
    print("Created blank 'en' model")

Created blank 'en' model


In [5]:
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe('ner', last=True)
else:
    ner = nlp.get_pipe('ner')

In [6]:
for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])
example = []
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in tqdm(TRAIN_DATA):
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotations)
            nlp.update(
                [example], 
                drop=0.5,  
                sgd=optimizer,
                losses=losses)
        print(losses)

100%|██████████| 15/15 [00:00<00:00, 49.28it/s]


{'ner': 89.95870500802994}


100%|██████████| 15/15 [00:00<00:00, 52.82it/s]


{'ner': 54.070997128263116}


100%|██████████| 15/15 [00:00<00:00, 50.85it/s]


...

{'ner': 1.5715079878512208}


100%|██████████| 15/15 [00:00<00:00, 52.45it/s]


{'ner': 0.42933476085960276}


100%|██████████| 15/15 [00:00<00:00, 52.27it/s]

{'ner': 4.198069516601132}


In [7]:
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)
pickle.dump(nlp, open( "aecNlp.pkl", "wb" ))

Saved model to ner


In [8]:
doc=nlp("Go to the northwest of the fountain and take a screenshot")
print(doc.ents)

(northwest, fountain, screenshot)


In [9]:
doc=nlp("professors said they wanna check the northeast windows of the apartment")
print(doc.ents)

(northeast, apartment)


In [9]:
doc=nlp("Check out southwest of C and screenshot")
print(doc.ents)

(southwest, C, screenshot)
